<a href="https://colab.research.google.com/github/brunobobadilla06/Portfolio-Proyectos/blob/main/Analisis_de_Turnos_Kamaleon_Therapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

np.random.seed(42)


In [ ]:
turnos = pd.read_csv("/content/turnos_kamaleon.csv")

turnos.head()


In [ ]:

turnos["fecha_inicio"] = pd.to_datetime(turnos["fecha_inicio"], errors="coerce")
turnos = turnos.dropna(subset=["fecha_inicio", "estado"])

turnos["estado"] = turnos["estado"].astype(str).str.strip().str.lower()
turnos["canal"] = turnos["canal"].astype(str).str.strip().str.lower()

estados_validos = {"asistio", "cancelado", "no_asistio"}
turnos = turnos[turnos["estado"].isin(estados_validos)]

turnos = turnos.drop_duplicates(subset=["id_turno"])

turnos.head()


In [ ]:

turnos["fecha"] = turnos["fecha_inicio"].dt.date
turnos["anio"] = turnos["fecha_inicio"].dt.year
turnos["mes"] = turnos["fecha_inicio"].dt.month
turnos["dia_semana_num"] = turnos["fecha_inicio"].dt.dayofweek  # 0=Lunes 1=Martes ... 6= Domingo
turnos["hora"] = turnos["fecha_inicio"].dt.hour

turnos["es_fin_de_semana"] = (turnos["dia_semana_num"] >= 5).astype(int)
turnos["es_horario_pico"] = ((turnos["hora"] >= 17) & (turnos["hora"] <= 21)).astype(int)

turnos["objetivo_no_asistio"] = (turnos["estado"] == "no_asistio").astype(int)

turnos["flag_asistio"] = (turnos["estado"] == "asistio").astype(int)
turnos["flag_cancelado"] = (turnos["estado"] == "cancelado").astype(int)
turnos["flag_no_asistio"] = (turnos["estado"] == "no_asistio").astype(int)

turnos.head()


In [ ]:

turnos["semana"] = turnos["fecha_inicio"].dt.to_period("W").dt.start_time

semanal = (
    turnos.groupby("semana", as_index=False)
          .agg(
              turnos_agendados=("id_turno", "count"),
              turnos_asistidos=("flag_asistio", "sum"),
              turnos_cancelados=("flag_cancelado", "sum"),
              turnos_no_asistio=("flag_no_asistio", "sum")
          )
)

semanal["tasa_asistencia"] = semanal["turnos_asistidos"] / semanal["turnos_agendados"]
semanal["tasa_no_asistencia"] = semanal["turnos_no_asistio"] / semanal["turnos_agendados"]
semanal["tasa_cancelacion"] = semanal["turnos_cancelados"] / semanal["turnos_agendados"]

semanal.tail()


In [ ]:

plt.figure()
plt.plot(semanal["semana"], semanal["turnos_agendados"])
plt.title("Demanda semanal de turnos (turnos agendados)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(semanal["semana"], semanal["tasa_no_asistencia"])
plt.title("Tasa de no-asistencia semanal")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

tabla_heatmap = (
    turnos.groupby(["dia_semana_num", "hora"], as_index=False)
          .agg(turnos=("id_turno", "count"))
          .pivot(index="dia_semana_num", columns="hora", values="turnos")
          .fillna(0)
)

plt.figure(figsize=(12, 4))
sns.heatmap(tabla_heatmap, cmap="viridis")
plt.title("Heatmap de demanda: Día de semana vs Hora")
plt.tight_layout()
plt.show()


In [ ]:

semanal["pronostico_media_movil_4"] = (
    semanal["turnos_agendados"]
    .shift(1)
    .rolling(4)
    .mean()
)

plt.figure()
plt.plot(semanal["semana"], semanal["turnos_agendados"], label="Real")
plt.plot(semanal["semana"], semanal["pronostico_media_movil_4"], label="Baseline: Media móvil 4")
plt.title("Real vs Baseline (media móvil 4)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

datos_forecast = semanal.dropna().copy()
datos_forecast["num_semana"] = datos_forecast["semana"].dt.isocalendar().week.astype(int)
datos_forecast["mes"] = datos_forecast["semana"].dt.month
datos_forecast["tendencia"] = np.arange(len(datos_forecast))

X_forecast = datos_forecast[["num_semana", "mes", "tendencia", "tasa_no_asistencia", "tasa_cancelacion"]]
y_forecast = datos_forecast["turnos_agendados"]

tscv = TimeSeriesSplit(n_splits=4)

maes = []
mapes = []
predicciones_oof = np.full(len(datos_forecast), np.nan)

for idx_train, idx_test in tscv.split(X_forecast):
    X_train, X_test = X_forecast.iloc[idx_train], X_forecast.iloc[idx_test]
    y_train, y_test = y_forecast.iloc[idx_train], y_forecast.iloc[idx_test]

    modelo_forecast = RandomForestRegressor(n_estimators=400, random_state=42)
    modelo_forecast.fit(X_train, y_train)

    pred = modelo_forecast.predict(X_test)
    predicciones_oof[idx_test] = pred

    maes.append(mean_absolute_error(y_test, pred))
    mapes.append(mean_absolute_percentage_error(y_test, pred))

mae_prom = float(np.mean(maes))
mape_prom = float(np.mean(mapes))

mae_prom, mape_prom


In [ ]:

plt.figure()
plt.plot(datos_forecast["semana"], y_forecast, label="Real")
plt.plot(datos_forecast["semana"], predicciones_oof, label="Predicción (validación temporal)")
plt.title(f"Forecast semanal - MAE: {mae_prom:.2f} | MAPE: {mape_prom:.2%}")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

columnas_modelo = [
    "canal",
    "hora",
    "dia_semana_num",
    "es_fin_de_semana",
    "es_horario_pico",
    "duracion_min",
    "primera_vez",
    "recordatorio_enviado"
]

columnas_modelo = [c for c in columnas_modelo if c in turnos.columns]

X = turnos[columnas_modelo].copy()
y = turnos["objetivo_no_asistio"].copy()

if "canal" in X.columns:
    X = pd.get_dummies(X, columns=["canal"], drop_first=True)

X = X.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelo_no_asistio = LogisticRegression(max_iter=1000)
modelo_no_asistio.fit(X_train, y_train)

proba_no_asistio = modelo_no_asistio.predict_proba(X_test)[:, 1]
pred_no_asistio = (proba_no_asistio >= 0.5).astype(int)

auc = roc_auc_score(y_test, proba_no_asistio)
reporte = classification_report(y_test, pred_no_asistio)
matriz = confusion_matrix(y_test, pred_no_asistio)

print("AUC:", auc)
print("\nReporte de clasificación:\n", reporte)
print("\nMatriz de confusión:\n", matriz)


In [ ]:

coeficientes = pd.Series(modelo_no_asistio.coef_[0], index=X_train.columns).sort_values()

plt.figure(figsize=(10,4))
coeficientes.tail(10).plot(kind="barh")
plt.title("Top variables asociadas a mayor probabilidad de no-asistencia (LogReg)")
plt.tight_layout()
plt.show()
